# HuggingFace

Hugging Face offers everything from tokenizers, which help computers make sense of text, to a huge variety of ready-to-go language models, and even a treasure trove of data suited for language tasks.

HF provides many things, some of which are:
1. Tokenizers
2. Models
3. Datasets
4. Trainers

**Tokenizers:** These work like a translator, converting the words we use into smaller parts and creating a secret code that computers can understand and work with.

**Models:** These are like the brain for computers, allowing them to learn and make decisions based on information they've been fed.

**Datasets:** Think of datasets as textbooks for computer models. They are collections of information that models study to learn and improve.

**Trainers:** Trainers are the coaches for computer models. They help these models get better at their tasks by practicing and providing guidance. HuggingFace Trainers implement the PyTorch training loop for you, so you can focus instead on other aspects of working on the model.



## Tokenizers

HuggingFace tokenizers help us break down text into smaller, manageable pieces called tokens. These tokenizers are easy to use and also remarkably fast due to their use of the Rust programming language.

**Tokenization:** The process by which an input series of characters is transformed into units the model is prepared to predict upon. A model trained on data tokenized by one tokenizer must use that same tokenizer for prediction; this is similar to feature engineering in traditional machine learning. It's like cutting a sentence into individual pieces, such as words or characters, to make it easier to analyze.

**Tokens:** Fundamental unit of input to language models. These are the pieces you get after cutting up text during tokenization, kind of like individual Lego blocks that can be words, parts of words, or even single letters. These tokens are converted to numerical values for models to understand.

**Pre-trained Model:** This is a ready-made model that has been previously taught with a lot of data.

**Uncased:** This means that the model treats uppercase and lowercase letters as the same.

In [1]:
from transformers import BertTokenizer

In [2]:
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
# See how many tokens are in the vocabulary
tokenizer.vocab_size

30522

In [4]:
# Tokenize the sentence
sent_0 = "I heart Generative AI"
tokens = tokenizer.tokenize(sent_0)

In [5]:
# Print the tokens
print(tokens)

['i', 'heart', 'genera', '##tive', 'ai']


In [6]:
# Show the token ids assigned to each token
print(tokenizer.convert_tokens_to_ids(tokens))

[1045, 2540, 11416, 6024, 9932]


In [7]:
dict(zip(tokens,tokenizer.convert_tokens_to_ids(tokens)))

{'i': 1045, 'heart': 2540, 'genera': 11416, '##tive': 6024, 'ai': 9932}

## Models

Hugging Face models provide a quick way to get started using models trained by the community. With only a few lines of code, you can load a pre-trained model and start using it on tasks such as sentiment analysis.

In [8]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

In [9]:
# Load a pre-trained sentiment analysis model
model_name = "textattack/bert-base-uncased-imdb"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [10]:
# Tokenize the input sequence
sent = "I love mathematics"
inputs = tokenizer.tokenize(text=sent)
dict(zip(inputs,tokenizer.convert_tokens_to_ids(inputs)))

{'i': 1045, 'love': 2293, 'mathematics': 5597}

In [11]:
# Make prediction
def use_model(input_text):
    inputs = tokenizer(
        text=input_text,
        return_tensors="pt")
    # Get predictions without updating the model - the no_grad method means no updating of the gradients
    with torch.no_grad():
        outputs = model(**inputs).logits
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        predicted_class = torch.argmax(probabilities)
    if predicted_class == 1:
        print(f"Sentiment: Positive ({probabilities[0][1] * 100:.2f}%)")
    else:
        print(f"Sentiment: Negative ({probabilities[0][0] * 100:.2f}%)")
    label = model.config.id2label[predicted_class.item()]
    arg_ind = predicted_class.item()
    print(f"\tModel label: {label}")
    print(f"\tModel arg_index: {arg_ind}")

In [12]:
use_model(sent)

Sentiment: Positive (82.51%)
	Model label: LABEL_1
	Model arg_index: 1


In [13]:
# Alternatively:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [14]:
use_model(sent)

Sentiment: Positive (82.51%)
	Model label: LABEL_1
	Model arg_index: 1


In [15]:
# Alternatively:
from transformers import pipeline
pipe = pipeline("text-classification", model="textattack/bert-base-uncased-imdb")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Device set to use mps:0


In [16]:
out = pipe(sent)
print(out[0])
print()
label = out[0]['label']
score = out[0]['score']
if label == "LABEL_1":
    print(f"Sentiment: Positive ({score * 100:.2f}%)")
else:
    print(f"Sentiment: Negative ({score * 100:.2f}%)")
# label = model.config.id2label[predicted_class.item()]
# arg_ind = predicted_class.item()
print(f"\tModel label: {label}")
print(f"\tModel score: {score}")

{'label': 'LABEL_1', 'score': 0.8251070380210876}

Sentiment: Positive (82.51%)
	Model label: LABEL_1
	Model score: 0.8251070380210876


## Datasets

HuggingFace Datasets library is a powerful tool for managing a variety of data types, like text and images, efficiently and easily. This resource is incredibly fast and doesn't use a lot of computer memory, making it great for handling big projects without any hassle.

**IMDb dataset:** A dataset of movie reviews that can be used to train a machine learning model to understand human sentiments.

**Apache Arrow:** A software framework that allows for fast data processing

In [25]:
# %pip install datasets huggingface_hub
import huggingface_hub
from datasets import load_dataset
from IPython.display import HTML, display

In [59]:
ds_list = huggingface_hub.list_datasets()

In [55]:
ds_list

<generator object HfApi.list_datasets at 0x3dde87ae0>

In [60]:
lst_ds = list(ds_list)

In [62]:
lst_ds[0]

DatasetInfo(id='agibot-world/AgiBotWorld-Alpha', author='agibot-world', sha='ea50c7f9e34593f99d9de76f90ac6c0623a0be79', created_at=datetime.datetime(2024, 12, 19, 9, 37, 11, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2025, 1, 7, 5, 0, 18, tzinfo=datetime.timezone.utc), private=False, gated='auto', disabled=False, downloads=7396, downloads_all_time=None, likes=150, paperswithcode_id=None, tags=['task_categories:other', 'language:en', 'size_categories:10M<n<100M', 'format:webdataset', 'modality:text', 'library:datasets', 'library:webdataset', 'library:mlcroissant', 'region:us', 'real-world', 'dual-arm', 'Robotics manipulation'], trending_score=77, card_data=None, siblings=None)

In [63]:
# load the IMDB dataset, which contains movie reviews and sentiment labels (positive or negative)
ds = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [67]:
# Retrieve a single review
rev_num = 42
sample_rev = ds["train"][rev_num]
display(HTML(sample_rev["text"][:450] + "..."))

In [70]:
sample_rev.keys()

dict_keys(['text', 'label'])

In [72]:
if sample_rev["label"] == 1:
    print("Sentiment: Positive")
else:
    print("Sentiment: Negative")
# Sentiment: Negative

Sentiment: Negative


## Trainers

[HuggingFace trainers](https://huggingface.co/docs/transformers/v4.47.1/en/main_classes/trainer#trainer) offer a simplified approach to training generative AI models, making it easier to set up and run complex machine learning tasks. This tool wraps up the hard parts, like handling data and carrying out the training process, allowing us to focus on the big picture and achieve better outcomes with our AI endeavors.

**Truncating:** This refers to shortening longer pieces of text to fit a certain size limit.

**Padding:** Adding extra data to shorter texts to reach a uniform length for processing.

**Batches:** Batches are small, evenly divided parts of data that the AI looks at and learns from each step of the way.

**Batch Size:** The number of data samples that the machine considers in one go during training.

**Epochs:** A complete pass through the entire training dataset. The more epochs, the more the computer goes over the material to learn.

**Dataset Splits:** Dividing the dataset into parts for different uses, such as training the model and testing how well it works.



In [73]:
from transformers import (DistilBertForSequenceClassification,
    DistilBertTokenizer,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset

In [74]:
# https://huggingface.co/distilbert/distilbert-base-uncased
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)
# https://huggingface.co/docs/transformers/model_doc/distilbert#transformers.DistilBertForSequenceClassification
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
# https://huggingface.co/docs/transformers/model_doc/distilbert#transformers.DistilBertTokenizer

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [75]:
ds = load_dataset("imdb")
tokenized_datasets = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [76]:
training_args = TrainingArguments(
    per_device_train_batch_size=64,
    output_dir="./results",
    learning_rate=2e-5,
    num_train_epochs=3,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

In [77]:
trainer.train()

Step,Training Loss
500,0.249400
1000,0.138000


TrainOutput(global_step=1173, training_loss=0.18140956101413477, metrics={'train_runtime': 2333.48, 'train_samples_per_second': 32.141, 'train_steps_per_second': 0.503, 'total_flos': 9935054899200000.0, 'train_loss': 0.18140956101413477, 'epoch': 3.0})

In [ ]:
import multiprocessing

multiprocessing.cpu_count()